# Data Processing

In [3]:

# Read data from the first file
file1_path = '/home/zixuan11/qjx/OpenKE/benchmarks/FB15K237/entity2id.txt'


# Read data from the second file
file2_path = '/home/zixuan11/qjx/OpenKE/benchmarks/FB15K237/FB15k_mid2name.txt'

# Save the mapping as a JSON file
output_json_path = '/home/zixuan11/qjx/OpenKE/benchmarks/FB15K237/id2name.json'

# Initialize an empty dictionary
entity2id = {}
mid2name = {}
# Open the text document and read its lines
with open(file1_path, 'r') as file:
    # Skip the first line
    total_lines = int(file.readline().strip())
    # Read the rest of the lines
    for line in file:
        # Split the line into key and value using the tab character ('\t')
        key, value = line.strip().split('\t')    
        # Convert the value to an integer, assuming it contains integer values
        value = int(value)    
        # Add the key-value pair to the dictionary
        entity2id[key] = value
# Print the resulting dictionary
print(entity2id)

with open(file2_path, 'r') as file:
    # Read the rest of the lines
    for line in file:
        # Split the line into key and value using the tab character ('\t')
        key, value = line.strip().split('\t')    
        # Convert the value to an integer, assuming it contains integer values
        # Add the key-value pair to the dictionary
        mid2name[key] = value
# Print the resulting dictionary
print(mid2name)

{'/m/027rn': 0, '/m/06cx9': 1, '/m/017dcd': 2, '/m/06v8s0': 3, '/m/07s9rl0': 4, '/m/0170z3': 5, '/m/01sl1q': 6, '/m/044mz_': 7, '/m/0cnk2q': 8, '/m/02nzb8': 9, '/m/04nrcg': 10, '/m/02sdk9v': 11, '/m/07nznf': 12, '/m/014lc_': 13, '/m/02qyp19': 14, '/m/02d413': 15, '/m/0q9kd': 16, '/m/0184jc': 17, '/m/03q5t': 18, '/m/07y_7': 19, '/m/0gqng': 20, '/m/073hkh': 21, '/m/0b76d_m': 22, '/m/029j_': 23, '/m/014_x2': 24, '/m/012ljv': 25, '/m/0ds35l9': 26, '/m/05r4w': 27, '/m/015qsq': 28, '/m/02bjrlw': 29, '/m/04bdxl': 30, '/m/02s2ft': 31, '/m/09c7w0': 32, '/m/0rs6x': 33, '/m/079vf': 34, '/m/0d90m': 35, '/m/015zyd': 36, '/m/05vsxz': 37, '/m/06qgvf': 38, '/m/04ljl_l': 39, '/m/03qcfvw': 40, '/m/0grwj': 41, '/m/05sxg2': 42, '/m/0rh6k': 43, '/m/01lp8': 44, '/m/05d7rk': 45, '/m/027dtxw': 46, '/m/05hs4r': 47, '/m/01pbxb': 48, '/m/03qcq': 49, '/m/084w8': 50, '/m/01lxd4': 51, '/m/0f0y8': 52, '/m/08815': 53, '/m/05bnp0': 54, '/m/0g56t9t': 55, '/m/0160w': 56, '/m/02vk52z': 57, '/m/0dbpyd': 58, '/m/053y0s': 5

In [4]:
id2name = {}
for i in entity2id:
    id2name[entity2id[i]] = mid2name[i]
import json
with open(output_json_path,"w") as f:
    json.dump(id2name,f)

In [1]:
train2id_path = "/home/zixuan11/qjx/OpenKE/benchmarks/FB15K237/train2id.txt"
with open(train2id_path, 'r') as file:
    total_lines = int(file.readline().strip())
    lines = [[int(item) for item in line.strip().split(' ')] for line in file.readlines()]

find all the (s1,t1,r1,t1',t2,r2,t3) in the list

which satisfies 
exist (s1,t1,r1) and (t1,t2,r2)
not exist  (s1,t1',r1)  # because I need to edit (s1,t1,r1) to (s1,t1',r1)
exist s* st. (s*,t1',r1)  # to make sure the new subject is a related one
exist (t1',t3,r2)

find all the (s1,t1,r1,t2,r2)
which satisfies 
exist (s1,t1,r1) and (t1,t2,r2)

In [2]:
from tqdm import tqdm
def find_tuples_optimized(data):
    # Organize data in a dictionary based on the second element of each triple
    data_dict = {}
    for triple in data:
        key = triple[0]
        if key not in data_dict:
            data_dict[key] = []
        data_dict[key].append(triple)

    result_tuples = []

    # Check for pairs of triples using the organized dictionary
    for s1, t1, r1 in tqdm(data):
        key = t1
        if key in data_dict:
            for t1_prime, t2, r2 in data_dict[key]:
                result_tuples.append((s1, t1, r1, t2, r2))
                
    return result_tuples

In [3]:
result = find_tuples_optimized(lines)
# print(result)

100%|██████████| 272115/272115 [00:09<00:00, 27565.25it/s]


In [4]:
#(s1, t1, r1, t2, r2
result = list(set(result))
print(len(result))

23217575


I have (s1, t1, r1, t2, r2)

Now I need to find a t1' that satisfy: 
not exist  (s1,t1',r1)  # because I need to edit (s1,t1,r1) to (s1,t1',r1)
exist s*  st. (s*,t1',r1)  # to make sure the new subject is a related one
exist t3 st. (t1',t3, r2) exist

In [5]:
import random

In [12]:
def find_t1_prime(five_tuple,train_data,number):
    tail_dict = {}
    head_dict = {}
    coup_dict = {}
    pp_dict = {}
    tt_dict = {}
    
    for triple in train_data:
        key = triple[2]
        if key not in tail_dict:
            tail_dict[key] = set()
        tail_dict[key].add(triple[1])
        
        if key not in head_dict:
            head_dict[key] = set()
        head_dict[key].add(triple[0])
        
        if key not in coup_dict:
            coup_dict[key] = set()
        coup_dict[key].add((triple[0],triple[1]))
        
        key1 = (triple[1],triple[2])
        if key1 not in pp_dict:
            pp_dict[key1] = set()
        pp_dict[key1].add(triple[0])
        
        key2 = (triple[0],triple[2])
        if key2 not in tt_dict:
            tt_dict[key2] = set()
        tt_dict[key2].add(triple[1])
        

    result_tuples = []

    for s1, t1, r1, t2, r2 in tqdm(five_tuple[:number]):
        # for r2, find a t3 randomly
        if r2 not in head_dict:
            continue
        if len(head_dict[r2])==0:
            continue
        for t1_prime in head_dict[r2]:
            if [s1,t1_prime,r1] in train_data:
                continue
            if (t1_prime,r1) not in pp_dict:
                continue
            # find t3:
            t3 = random.sample(tt_dict[(t1_prime,r2)],1)[0]
            result_tuples.append((s1,t1,r1,t1_prime,t2,r2,t3))
            break # 对于一行有一个就行了 
            
    return result_tuples

In [15]:
result_tuples = find_t1_prime(result,lines,number=100)

  0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_18985/1486105084.py:47: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  t3 = random.sample(tt_dict[(t1_prime,r2)],1)[0]
100%|██████████| 100/100 [00:12<00:00,  8.20it/s]


In [20]:
with open("transe_edit_data.txt","w") as file:
    file.write("s1,t1,r1,t1',t2,r2,t3\n")
    for tpl in result_tuples:
        line = ' '.join(map(str,tpl))
        file.write(line+"\n")

In [34]:
import json
with open("/home/zixuan11/qjx/OpenKE/benchmarks/FB15K237/id2name.json","r") as json_file:
    id2name = json.load(json_file)
    
id2relation = {}
with open("/home/zixuan11/qjx/OpenKE/benchmarks/FB15K237/relation2id.txt", "r") as file:
    lines = file.readlines()
    for line in lines[1:]:
        numbers = line.strip('\n').split('\t')
        key = (int)(numbers[1])
        value = numbers[0]
        id2relation[key] = value

In [43]:
id2name

{'0': 'Dominican_Republic',
 '1': 'Republic',
 '2': 'Mighty_Morphin_Power_Rangers',
 '3': 'Wendee_Lee',
 '4': 'Drama',
 '5': 'American_History_X',
 '6': 'Michelle_Rodriguez',
 '7': 'Naveen_Andrews',
 '8': 'Australia_national_association_football_team',
 '9': 'Midfielder',
 '10': 'Maldives_national_football_team',
 '11': 'Forward',
 '12': 'Bryan_Singer',
 '13': 'Star_Trek_Nemesis',
 '14': 'BAFTA_Award_for_Best_Original_Screenplay',
 '15': 'Philadelphia',
 '16': 'Danny_DeVito',
 '17': 'Guy_Pearce',
 '18': 'harpsichord',
 '19': 'Violin',
 '20': 'Academy_Award_for_Best_Foreign_Language_Film',
 '21': '61st_Academy_Awards',
 '22': 'Anonymous',
 '23': 'DVD',
 '24': 'City_of_Angels',
 '25': 'Gabriel_Yared',
 '26': 'Bridesmaids',
 '27': 'Portugal',
 '28': 'Serpico',
 '29': 'Italian_Language',
 '30': 'Catherine_Keener',
 '31': 'Chris_Cooper',
 '32': 'United_States_of_America',
 '33': 'St._Augustine',
 '34': 'Stan_Lee',
 '35': 'X-Men',
 '36': 'California_Institute_of_the_Arts',
 '37': 'Chiwetel_E

In [56]:
import json
# (s1,t1,r1,t1',t2,r2,t3)
editing_data = []
for item in result_tuples:
    editing_data_item = dict()
    editing_data_item['original_triple'] = (id2name[str(item[0])],id2name[str(item[1])],id2relation[item[2]])
    editing_data_item['after_edit_object'] = id2name[str(item[3])]
    editing_data_item['before_preliminary'] = (id2name[str(item[1])],id2name[str(item[4])],id2relation[item[5]])
    editing_data_item['after_preliminary'] = (id2name[str(item[3])],id2name[str(item[6])],id2relation[item[5]])
    editing_data.append(editing_data_item)

In [63]:
with open("editing_data_text.json","w") as json_file:
    json.dump(editing_data,json_file)